In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

nlp_getting_started_path = kagglehub.competition_download('nlp-getting-started')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [ ]:
import re
import string

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import transformers
from transformers import BertTokenizer
from transformers import TFAutoModel
from sklearn.model_selection import train_test_split

print(tf.__version__)
print(transformers.__version__)


2025-06-10 15:30:13.766184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-10 15:30:13.766326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 15:30:14.107312: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0
4.42.3


In [ ]:
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
X = df.drop(['target'],axis=1)
y = df['target']
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
X_test = test_df.copy()

In [ ]:
ids = list(X_test['id'])
ids

[0,
 2,
 3,
 9,
 11,
 12,
 21,
 22,
 27,
 29,
 30,
 35,
 42,
 43,
 45,
 46,
 47,
 51,
 58,
 60,
 69,
 70,
 72,
 75,
 84,
 87,
 88,
 90,
 94,
 99,
 101,
 103,
 106,
 108,
 111,
 115,
 116,
 122,
 123,
 124,
 125,
 127,
 140,
 142,
 147,
 148,
 150,
 152,
 154,
 155,
 166,
 167,
 169,
 177,
 179,
 181,
 186,
 188,
 189,
 192,
 200,
 202,
 206,
 207,
 214,
 217,
 223,
 224,
 227,
 228,
 230,
 233,
 234,
 236,
 239,
 250,
 255,
 257,
 259,
 275,
 278,
 282,
 284,
 286,
 288,
 292,
 295,
 300,
 304,
 305,
 306,
 308,
 311,
 317,
 319,
 323,
 324,
 325,
 326,
 333,
 339,
 342,
 343,
 350,
 351,
 357,
 359,
 362,
 366,
 367,
 369,
 373,
 374,
 376,
 377,
 378,
 379,
 382,
 385,
 387,
 388,
 391,
 392,
 395,
 399,
 400,
 403,
 405,
 408,
 411,
 414,
 416,
 417,
 422,
 425,
 428,
 430,
 431,
 433,
 434,
 439,
 441,
 449,
 458,
 460,
 464,
 473,
 488,
 491,
 494,
 497,
 500,
 505,
 507,
 508,
 510,
 511,
 515,
 525,
 529,
 532,
 534,
 537,
 539,
 541,
 545,
 547,
 548,
 549,
 553,
 554,
 555,
 5

In [ ]:
#Use regex to clean the data
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def decontraction(text):
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)

    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)

    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)
    return text

def seperate_alphanumeric(text):
    words = text
    words = re.findall(r"[^\W\d_]+|\d+", words)
    return " ".join(words)

def cont_rep_char(text):
    tchr = text.group(0)

    if len(tchr) > 1:
        return tchr[0:2]

def unique_char(rep, text):
    substitute = re.sub(r'(\w)\1+', rep, text)
    return substitute

X['text'] = X['text'].apply(lambda x : remove_url(x))
X['text'] = X['text'].apply(lambda x : remove_punct(x))
X['text'] = X['text'].apply(lambda x : remove_emoji(x))
X['text'] = X['text'].apply(lambda x : decontraction(x))
X['text'] = X['text'].apply(lambda x : seperate_alphanumeric(x))
X['text'] = X['text'].apply(lambda x : unique_char(cont_rep_char,x))

X_test['text'] = X_test['text'].apply(lambda x : remove_url(x))
X_test['text'] = X_test['text'].apply(lambda x : remove_punct(x))
X_test['text'] = X_test['text'].apply(lambda x : remove_emoji(x))
X_test['text'] = X_test['text'].apply(lambda x : decontraction(x))
X_test['text'] = X_test['text'].apply(lambda x : seperate_alphanumeric(x))
X_test['text'] = X_test['text'].apply(lambda x : unique_char(cont_rep_char,x))

In [ ]:
seq_len = 256
batch_size = 16
num_samples = len(X)
model_name = 'google-bert/bert-base-uncased'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

train_tokens = tokenizer(
    X['text'].tolist(),
    max_length=seq_len,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='np'
)

labels = np.zeros((num_samples, y.max() + 1))
labels[np.arange(num_samples), y] = 1

dataset = tf.data.Dataset.from_tensor_slices(
    (
        train_tokens['input_ids'],
        train_tokens['attention_mask'],
        labels
    )
)

def map_func(input_ids, masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': masks
    }, labels

dataset = dataset.map(map_func)
dataset = dataset.shuffle(10000).batch(batch_size=batch_size, drop_remainder=True)

split = 0.6
train_size = int((train_tokens['input_ids'].shape[0] // batch_size) * 0.7)
val_size = int((train_tokens['input_ids'].shape[0] // batch_size) * 0.3)
train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size).take(val_size)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
ids

[0,
 2,
 3,
 9,
 11,
 12,
 21,
 22,
 27,
 29,
 30,
 35,
 42,
 43,
 45,
 46,
 47,
 51,
 58,
 60,
 69,
 70,
 72,
 75,
 84,
 87,
 88,
 90,
 94,
 99,
 101,
 103,
 106,
 108,
 111,
 115,
 116,
 122,
 123,
 124,
 125,
 127,
 140,
 142,
 147,
 148,
 150,
 152,
 154,
 155,
 166,
 167,
 169,
 177,
 179,
 181,
 186,
 188,
 189,
 192,
 200,
 202,
 206,
 207,
 214,
 217,
 223,
 224,
 227,
 228,
 230,
 233,
 234,
 236,
 239,
 250,
 255,
 257,
 259,
 275,
 278,
 282,
 284,
 286,
 288,
 292,
 295,
 300,
 304,
 305,
 306,
 308,
 311,
 317,
 319,
 323,
 324,
 325,
 326,
 333,
 339,
 342,
 343,
 350,
 351,
 357,
 359,
 362,
 366,
 367,
 369,
 373,
 374,
 376,
 377,
 378,
 379,
 382,
 385,
 387,
 388,
 391,
 392,
 395,
 399,
 400,
 403,
 405,
 408,
 411,
 414,
 416,
 417,
 422,
 425,
 428,
 430,
 431,
 433,
 434,
 439,
 441,
 449,
 458,
 460,
 464,
 473,
 488,
 491,
 494,
 497,
 500,
 505,
 507,
 508,
 510,
 511,
 515,
 525,
 529,
 532,
 534,
 537,
 539,
 541,
 545,
 547,
 548,
 549,
 553,
 554,
 555,
 5

In [ ]:
test_tokens = tokenizer(
    X_test['text'].tolist(),
    max_length=seq_len,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='np'
)


test_dataset = tf.data.Dataset.from_tensor_slices(
    (
        test_tokens['input_ids'],
        test_tokens['attention_mask']
    )
)

def map_func(input_ids, masks):
    return {
        'input_ids': input_ids,
        'attention_mask': masks
    }
test_dataset = test_dataset.map(map_func)
test_dataset = test_dataset.batch(batch_size=batch_size)
print(list(test_dataset.as_numpy_iterator())[0])

test_ds = test_dataset

{'input_ids': array([[  101,  2074,  3047, ...,     0,     0,     0],
       [  101,  2657,  2055, ...,     0,     0,     0],
       [  101,  2045,  2003, ...,     0,     0,     0],
       ...,
       [  101,  2054,  2065, ...,     0,     0,     0],
       [  101, 12476,   102, ...,     0,     0,     0],
       [  101,  6484, 17264, ...,     0,     0,     0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}


In [ ]:
model = TFAutoModel.from_pretrained(model_name, output_hidden_states=True)

# Two inputs||
input_ids = tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(seq_len,), name='attention_mask', dtype='int32')

# Transformer
# embeddings = model.bert(input_ids, attention_mask=mask)[1]
embeddings = model(input_ids, attention_mask=mask).last_hidden_state
# We are considering only the CLS token (the first token) for classification, as that is a good representation of the aggregate, and is commonly used for classification
# This is infact known as pooler output, this has a shape of [batch_size, 1, hidden_size]
embeddings = embeddings[:, 0, :]
# Classifier head
x = tf.keras.layers.Dense(512, activation='relu')(embeddings)
x = tf.keras.layers.Dropout(0.1)(x)
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(x)

bert_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# freeze bert layers
# bert_model.layers[2].trainable = False

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.BinaryAccuracy()

bert_model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

history = bert_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1,
    batch_size=batch_size
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

 21/332 [>.............................] - ETA: 4:02 - loss: 0.7194 - binary_accuracy: 0.5685

In [ ]:
embeddings_pool = model(input_ids, attention_mask=mask).pooler_output
print(embeddings_pool)
x = tf.keras.layers.Dense(512, activation='relu')(embeddings_pool)
x = tf.keras.layers.Dropout(0.1)(x)
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(x)

bert_model_pooler = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.BinaryAccuracy()

bert_model_pooler.compile(optimizer=optimizer, loss=loss, metrics=[acc])

history = bert_model_pooler.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1,
    batch_size=batch_size
)

In [ ]:
embeddings = model(input_ids, attention_mask=mask).last_hidden_state
embeddings_averaged = embeddings[:,0,:]
for i in range(1,256):
    embeddings_averaged += embeddings[:,i,:]
embeddings_averaged = embeddings_averaged/256
print(embeddings_averaged.shape)
x = tf.keras.layers.Dense(512, activation='relu')(embeddings_averaged)
x = tf.keras.layers.Dropout(0.1)(x)
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(x)

bert_model_averaged = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.BinaryAccuracy()
optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=0.1)
bert_model_averaged.compile(optimizer=optimizer, loss=loss, metrics=[acc])
history = bert_model_averaged.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1,
    batch_size=batch_size
)

In [ ]:
predictions_CLS = bert_model.predict(test_ds)
predictions_avg = bert_model_averaged.predict(test_ds)
predictions_pooler = bert_model_pooler.predict(test_ds)

In [ ]:
predicted_list = []
for prediction in predictions_CLS:
    if prediction[0] > prediction[1]:
        predicted_list.append(0)
    else:
        predicted_list.append(1)
rdf = pd.DataFrame({'id':ids, 'target': predicted_list})
rdf.to_csv('submission_4_CLS.csv',index=False)
predicted_list = []
for prediction in predictions_avg:
    if prediction[0] > prediction[1]:
        predicted_list.append(0)
    else:
        predicted_list.append(1)
rdf = pd.DataFrame({'id':ids, 'target': predicted_list})
rdf.to_csv('submission_4_avg.csv',index=False)
predicted_list = []
for prediction in predictions_pooler:
    if prediction[0] > prediction[1]:
        predicted_list.append(0)
    else:
        predicted_list.append(1)
rdf = pd.DataFrame({'id':ids,  'target': predicted_list})
rdf.to_csv('submission_4_pool.csv',index=False)